In [1]:
import nltk
from nltk import Tree
from nltk.corpus import treebank
from nltk import Nonterminal, Production
from nltk import induce_pcfg

In [8]:
print("Induce PCFG grammar from treebank data:")

productions = []
Leaves=set()
for item in treebank.fileids():
    for tree in treebank.parsed_sents(item):
        tree.collapse_unary(collapsePOS = False)# Remove branches A-B-C into A-B+C
        tree.chomsky_normal_form(horzMarkov = 2)# Remove A->(B,C,D) into A->B,C+D->D
        productions += tree.productions()
        for i in range(len(tree.leaves())):
            Leaves.add(tree.leaves()[i])

Induce PCFG grammar from treebank data:


In [9]:
Leaves=list(Leaves)

In [11]:
from nltk  import Nonterminal
S = Nonterminal('S')
grammar = induce_pcfg(S, productions)

In [14]:
from nltk.parse import ViterbiParser
from functools import reduce
sentence="I saw it"
tokens = sentence.split(' ')
parser = ViterbiParser(grammar)
parses = parser.parse_all(tokens)

if parses: 
    lp = len(parses)
    p = reduce(lambda a,b:a+b.prob(), parses, 0.0)
else:
    p = 0
for parse in parses:
    print(parse)

(S (NP-SBJ (PRP I)) (VP (VBD saw) (NP (PRP it)))) (p=5.31475e-10)


In [ ]:
count = {}
rule_count = {}
rule_probs = {}
for rule in grammar:
    if rule[0] not in count:
        count[rule[0]] = 1
    else:
        count[rule[0]] += 1
for rule in grammar:
    if rule not in rule_count:
        rule_count[rule] = 1
    else:
        rule_count[rule] += 1
for entry in rule_count:
    rule_probs[entry] = float(rule_count[entry])/count[entry[0]]

In [14]:
grammar_rules=[]
for line in productions:
    grammar_rules.append(tuple(str(line).rstrip('\n').split(' -> ')))

In [15]:
count = {}
rule_count = {}
rule_probs = {}
for rule in grammar_rules:
    if rule[0] not in count:
        count[rule[0]] = 1
    else:
        count[rule[0]] += 1
for rule in grammar_rules:
    if rule not in rule_count:
        rule_count[rule] = 1
    else:
        rule_count[rule] += 1
for entry in rule_count:
    rule_probs[entry] = float(rule_count[entry])/count[entry[0]]

In [20]:
non_terms = set()
for rules in set(grammar_rules):
    non_terms.add(rules[0])
non_terms=list(non_terms)
sent="I saw John with my eyes"
sent=sent.split(" ")
print(sent)

['I', 'saw', 'John', 'with', 'my', 'eyes']


In [22]:
score=[[[0.0 for i in range(len(non_terms))] for j in range(len(sent)+1)] for k in range(len(sent)+1)]
back =[[[-1 for i in range(len(non_terms))] for j in range(len(sent)+1)] for k in range(len(sent)+1)]

In [32]:
rule_index = {}
from tqdm import tqdm_notebook as tqdm
for i,word in tqdm(enumerate(sent)):
    rules_used = []
    rules_not_used = []
    for j,A in tqdm(enumerate(non_terms)):
        r = A, '\'' + word + '\''
        if r in grammar_rules:
            score[i][i+1][j] = rule_probs[r]
            rules_used.append(j)
            rule_index[A] = j
        else:
            rules_not_used.append(j)
            rule_index[A] = j
    rules_used_temp = rules_used[:]
    rules_not_used_temp = rules_not_used[:]
    added = True
    while added:
        print(added)
        added = False
        for a in rules_not_used:
            for b in rules_used:
                r = non_terms[a], non_terms[b]
                if r in grammar_rules:
                    prob = rule_probs[r] * score[i][i+1][b]
                    if prob > score[i][i+1][a]:
                        score[i][i+1][a] = prob
                        back[i][i+1][a] = b
                        rules_used_temp.append(a)
                        try:
                            rules_not_used_temp.remove(a)
                            rules_used_temp.remove(b)
                        except ValueError:
                            pass
                        added = True

        rules_used = rules_used_temp[:]
        rules_not_used =rules_not_used_temp[:]

    

True


True


True


True
True


True
True


True
True


In [34]:
bin_set = set()
for rules in grammar_rules:
    if len(rules[1].split(' ')) == 2:
        b, c = rules[1].split(' ')
        bin_set.add((rules[0],b,c))
        bin_set.add((rules[0], c, b))

In [ ]:
binary_rules =list(bin_set)
for span in tqdm(range(2,len(sent)+1)):
    print("span:"+str(span))
    for begin in tqdm(range(len(sent)+1-span)):
        rules_used = []
        end = begin + span
        print("span: "+str(span)+" begin: "+str(begin)+" end: "+str(end))
        for split in tqdm(range(begin+1, end)):
            print("span: "+str(span)+" begin: "+str(begin)+" end: "+str(end)+" split: "+str(split))
            for rule in tqdm(binary_rules):
                a, b, c = rule_index[rule[0]], rule_index[rule[1]], rule_index[rule[2]]
                concat_rule = rule[0], ' '.join((rule[1], rule[2]))
                if concat_rule in grammar_rules:
                    prob = score[begin][split][b] * score[split][end][c] * rule_probs[concat_rule]
                else:
                    continue
                if prob > score[begin][end][a]:
                    score[begin][end][a] = prob
                    back[begin][end][a] = split, b, c
                    rules_used.append(a)

            ### Handle Unaries
        added = True
        while added:
            added = False
            for a in range(len(non_terms)):
                for b in rules_used:
                    r = non_terms[a], non_terms[b]
                    if r in grammar_rules:
                        prob = rule_probs[r] * score[begin][end][b]
                        if prob > score[begin][end][a]:
                            score[begin][end][a] = prob
                            back[begin][end][a] = b
                            added = True

span:2


span: 2 begin: 0 end: 2


span: 2 begin: 0 end: 2 split: 1


span: 2 begin: 1 end: 3


span: 2 begin: 1 end: 3 split: 2


span: 2 begin: 2 end: 4


span: 2 begin: 2 end: 4 split: 3


In [26]:
from collections import defaultdict
print(grammar.productions()[0].lhs())
source=defaultdict()
prob=defaultdict()
rhs_1=set()
lhs_1=set()
total=set()
for i in range(len(grammar.productions())):
    q=len(grammar.productions()[i].rhs())
    print(grammar.productions()[i].lhs())
    A=str(grammar.productions()[i]).split(' -> ')[0]
    print(A)
    BC=str(grammar.productions()[i]).split(' -> ')[1].split()[:-1]
    source[A]=BC
    lhs_1.add(A)
    total.add(A)
    p1=float(str(grammar.productions()[i]).split(' -> ')[1].split()[-1][1:-1])
    if len(BC)==1:
        prob[str(grammar.productions()[i]).split(' -> ')[0]+" -> "+str(BC[0])]=p1
        rhs_1.add(BC[0])
        total.add(BC[0])
    elif len(BC)==2:
        prob[str(grammar.productions()[i]).split(' -> ')[0]+" -> "+str(BC[0])+" "+str(BC[1])]=p1
        rhs_1.add(BC[0])
        rhs_1.add(BC[1])
        total.add(BC[0])
        total.add(BC[1])
rhs_dict=defaultdict()
lhs_dict=defaultdict()
rhs_dict_r=defaultdict()
lhs_dict_l=defaultdict()
for i, item in enumerate(lhs_1):
    lhs_dict[item]=i
    lhs_dict_l[i]=item
total_dict=defaultdict()
reverse_dict=defaultdict()
for i, item in enumerate(total):
    total_dict[item]=i
    reverse_dict[i]=item

S
S
S
NP-SBJ
NP-SBJ
NP
NP
NNP
NNP
NNP
NNP
NP-SBJ|<,-ADJP>
NP-SBJ|<,-ADJP>
,
,
NP-SBJ|<ADJP-,>
NP-SBJ|<ADJP-,>
ADJP
ADJP
NP
NP
CD
CD
NNS
NNS
JJ
JJ
S|<VP-.>
S|<VP-.>
VP
VP
MD
MD
VP
VP
VB
VB
VP|<NP-PP-CLR>
VP|<NP-PP-CLR>
NP
NP
DT
DT
NN
NN
VP|<PP-CLR-NP-TMP>
VP|<PP-CLR-NP-TMP>
PP-CLR
PP-CLR
IN
IN
NP
NP
DT
DT
NP|<JJ-NN>
NP|<JJ-NN>
JJ
JJ
NN
NN
NP-TMP
NP-TMP
NNP
NNP
CD
CD
.
.
NP-SBJ
NP-SBJ
NNP
NNP
VP
VP
VBZ
VBZ
NP-PRD
NP-PRD
NP
NP
NN
NN
PP
PP
IN
IN
NP
NP
NNP
NNP
NNP
NNP
NP|<,-NP>
NP|<,-NP>
NP
NP
NP|<NNP-VBG>
NP|<NNP-VBG>
NNP
NNP
NP|<VBG-NN>
NP|<VBG-NN>
VBG
VBG
NN
NN
S
S
NP-SBJ-1
NP-SBJ-1
NNP
NNP
NNP
NNP
NP-SBJ-1|<,-UCP>
NP-SBJ-1|<,-UCP>
NP-SBJ-1|<UCP-,>
NP-SBJ-1|<UCP-,>
UCP
UCP
CD
CD
UCP|<CC-NP>
UCP|<CC-NP>
CC
CC
NP
NP
NP
NP
JJ
JJ
NP
NP
NNP
NNP
NP|<NNP-NNP>
NP|<NNP-NNP>
NNP
NNP
NP|<NNP-NNP>
NP|<NNP-NNP>
NNP
NNP
NNP
NNP
VP
VP
VBD
VBD
VP
VP
VBN
VBN
S
S
NP-SBJ
NP-SBJ
-NONE-
-NONE-
NP
NP
DT
DT
NP|<JJ-JJ>
NP|<JJ-JJ>
JJ
JJ
JJ
JJ
NN
NN
S
S
S-TPC-1
S-TPC-1
NP-SBJ
NP-SBJ
DT
DT
NN
NN
NN
NN
RRC
RRC
ADVP

NP
NN
NN
NN
NN
NP-TMP
NP-TMP
NP-SBJ-1
NP-SBJ-1
NNS
NNS
NP
NP
JJ
JJ
JJ
JJ
NNS
NNS
VP
VP
VBD
VBD
VP|<S-,>
VP|<S-,>
VP
VP
VB
VB
VP|<,-PP-LOC>
VP|<,-PP-LOC>
IN
IN
NP
NP
NNS
NNS
NN
NN
VBP
VBP
JJ
JJ
NNS
NNS
NN
NN
NNS
NNS
JJ
JJ
NP|<JJ-NN>
NP|<JJ-NN>
JJ
JJ
NN
NN
NN
NN
NP
NP
NP|<CD-JJ>
NP|<CD-JJ>
CD
CD
JJ
JJ
VBN
VBN
NP-LGS
NP-LGS
NNP
NNP
NP-LGS|<NNP-NNP>
NP-LGS|<NNP-NNP>
NNP
NNP
NNP
NNP
NNP
NNP
VP
VP
VBD
VBD
VP|<NP-ADV-PP-DIR>
VP|<NP-ADV-PP-DIR>
NP-ADV
NP-ADV
NN
NN
NN
NN
VP|<PP-DIR-PP-DIR>
VP|<PP-DIR-PP-DIR>
NP
NP
CD
CD
NN
NN
CD
CD
NN
NN
VP
VP
VBD
VBD
NP-TMP
NP-TMP
NNP
NNP
NN
NN
NNS
NNS
VP
VP
VBP
VBP
UCP
UCP
NN
NN
NNS
NNS
UCP|<CC-SBAR>
UCP|<CC-SBAR>
NP-SBJ
NP-SBJ
NP-SBJ|<JJ-NN>
NP-SBJ|<JJ-NN>
JJ
JJ
VP
VP
VBZ
VBZ
JJ
JJ
NN
NN
NP
NP
NP
NP
NP|<NNS-POS>
NP|<NNS-POS>
POS
POS
NNS
NNS
VP
VP
VBD
VBD
VP|<PP-EXT-PP-DIR>
VP|<PP-EXT-PP-DIR>
PP-EXT
PP-EXT
NN
NN
VP|<PP-DIR-PP>
VP|<PP-DIR-PP>
CD
CD
NNS
NNS
NP
NP
JJS
JJS
IN
IN
NP
NP
NNP
NNP
NNP
NNP
NP-SBJ-1
NP-SBJ-1
JJR
JJR
NNS
NNS
VBN
VBN
VP
VP
VB
VB
VP|<NP-SB

NNP
NNP
NNP
NNP
VP
VP
VBD
VBD
VP|<ADVP-DIR-CLR-PP-CLR>
VP|<ADVP-DIR-CLR-PP-CLR>
ADVP-DIR-CLR
ADVP-DIR-CLR
IN
IN
VP|<PP-CLR-,>
VP|<PP-CLR-,>
NN
NN
NNP
NNP
NNP
NNP
NNP
NNP
VP
VP
VBG
VBG
S
S
NNS
NNS
VP
VP
ADJP-PRD-3
ADJP-PRD-3
RB
RB
ADJP-PRD-3|<JJ-S>
ADJP-PRD-3|<JJ-S>
-NONE-
-NONE-
S|<CC-S>
S|<CC-S>
S|<S-S-4>
S|<S-S-4>
S
S
NP-SBJ=2
NP-SBJ=2
NP-SBJ=2|<JJ-NN>
NP-SBJ=2|<JJ-NN>
JJ
JJ
NN
NN
ADVP-PRD-LOC=3
ADVP-PRD-LOC=3
ADVP-PRD-LOC=3|<RB-PP-LOC>
ADVP-PRD-LOC=3|<RB-PP-LOC>
ADVP-PRD-LOC=3|<PP-LOC-S>
ADVP-PRD-LOC=3|<PP-LOC-S>
NN
NN
S-4
S-4
VB
VB
NN
NN
NN
NN
VP
VP
VBZ
VBZ
NNP
NNP
NNP
NNP
NNP
NNP
NNP
NNP
NP
NP
NP|<VBG-JJ>
NP|<VBG-JJ>
JJ
JJ
NNS
NNS
NNP
NNP
RB
RB
VBN
VBN
NP
NP
JJ
JJ
NN
NN
NP|<PP-LOC-SBAR>
NP|<PP-LOC-SBAR>
NP
NP
NN
NN
CD
CD
NN
NN
NNS
NNS
SBAR
SBAR
WHPP-2
WHPP-2
WHNP
WHNP
VP
VP
JJ
JJ
PP-LOC
PP-LOC
S
S
NP-SBJ-4
NP-SBJ-4
NP-SBJ-4|<,-VP>
NP-SBJ-4|<,-VP>
NP-SBJ-4|<VP-,>
NP-SBJ-4|<VP-,>
NNP
NNP
VP
VP
VP|<NP-S-PRP>
VP|<NP-S-PRP>
CD
CD
VP|<S-PRP-,>
VP|<S-PRP-,>
S-PRP
S-PRP
VB
VB
VP|<,-PP-1>


NN
VB
VB
NN
NN
S
S
NP-SBJ-21
NP-SBJ-21
-NONE-
-NONE-
NN
NN
NNP
NNP
NNP
NNP
NNP
NNP
NP
NP
NNP
NNP
NP|<,-NP-LOC>
NP|<,-NP-LOC>
NNP
NNP
NP-PRD
NP-PRD
NP-PRD|<NN-NN>
NP-PRD|<NN-NN>
NN
NN
S
S
S|<S-CC>
S|<S-CC>
S
S
PRP
PRP
S|<RB-VP>
S|<RB-VP>
RB
RB
VBP
VBP
VB
VB
S|<S-.>
S|<S-.>
VP
VP
VBP
VBP
VP
VP
IN
IN
NN
NN
VBP
VBP
NP
NP
NP
NP
JJ
JJ
JJ
JJ
NN
NN
NN
NN
NP-SBJ
NP-SBJ
NP-SBJ|<NNP-NNS>
NP-SBJ|<NNP-NNS>
NNP
NNP
VP
VP
VBP
VBP
VP|<NP-2-PP-CLR>
VP|<NP-2-PP-CLR>
NP-2
NP-2
NNS
NNS
NP-2|<,-SBAR>
NP-2|<,-SBAR>
NP-2|<SBAR-,>
NP-2|<SBAR-,>
S
S
NP-SBJ-22
NP-SBJ-22
-NONE-
-NONE-
NN
NN
NN
NN
VP
VP
VBG
VBG
ADJP
ADJP
ADJP|<JJ-CC>
ADJP|<JJ-CC>
JJ
JJ
ADJP|<CC-JJ>
ADJP|<CC-JJ>
JJ
JJ
VB
VB
JJ
JJ
NN
NN
NN
NN
NN
NN
VB
VB
CD
CD
NNS
NNS
PRN
PRN
ADJP
ADJP
ADJP
ADJP
RB
RB
ADJP|<RB-JJ>
ADJP|<RB-JJ>
NP
NP
NP|<JJS-JJ>
NP|<JJS-JJ>
VB
VB
JJ
JJ
NP|<,-PP-DIR>
NP|<,-PP-DIR>
PP-DIR
PP-DIR
PP-DIR|<IN-NP>
PP-DIR|<IN-NP>
SBAR
SBAR
SBAR
SBAR
WHNP-23
WHNP-23
-NONE-
-NONE-
QP
QP
RB
RB
JJ
JJ
SBAR|<CC-SBAR>
SBAR|<CC-SBAR>
SBAR
SBAR
WHN

NN
NN
NN
NNS
NNS
NP
NP
NN
NN
NN
NN
VP
VP
VP|<ADVP-PRD-LOC-PP-TMP>
VP|<ADVP-PRD-LOC-PP-TMP>
ADVP-PRD-LOC
ADVP-PRD-LOC
S
S
S|<NP-SBJ-PRN>
S|<NP-SBJ-PRN>
NP-SBJ
NP-SBJ
NP-SBJ|<JJ-CD>
NP-SBJ|<JJ-CD>
PP
PP
JJ
JJ
NP
NP
JJ
JJ
NNS
NNS
NP
NP
NNP
NNP
NNP
NNP
NNP
NNP
NNS
NNS
VP
VP
VB
VB
NNS
NNS
JJ
JJ
ADJP-PRD
ADJP-PRD
JJR
JJR
NN
NN
NN
NN
ADJP
ADJP
QP
QP
JJR
JJR
NNS
NNS
NP-PRD
NP-PRD
NP-PRD|<NN-NN>
NP-PRD|<NN-NN>
NN
NN
NP-PRD|<NN-NNS>
NP-PRD|<NN-NNS>
NN
NN
NNS
NNS
S
S
NP-SBJ
NP-SBJ
NNP
NNP
NNP
NNP
NNP
NNP
S|<PP-TMP-VP>
S|<PP-TMP-VP>
CD
CD
VBD
VBD
JJ
JJ
NN
NN
VP
VP
VBD
VBD
VP|<NP-PRD-SBAR-TMP>
VP|<NP-PRD-SBAR-TMP>
NN
NN
SBAR-TMP
SBAR-TMP
NP-ADV
NP-ADV
SBAR-TMP|<IN-S>
SBAR-TMP|<IN-S>
NNP
NNP
VBD
VBD
NNS
NNS
CD
CD
NNP
NNP
NNP
NNP
NP
NP
NP|<NP-PP>
NP|<NP-PP>
NNP
NNP
NNP
NNP
SBAR
SBAR
WHNP-32
WHNP-32
-NONE-
-NONE-
NN
NN
NNS
NNS
NNP
NNP
NNP
NNP
NNP
NNP
NNP
NNP
NP
NP
NNP
NNP
NNS
NNS
NNS
NNS
NNS
NNS
SBAR
SBAR
WHNP-33
WHNP-33
-NONE-
-NONE-
VBP
VBP
VB
VB
IN
IN
NN
NN
NN
NN
VB
VB
VP|<NP-PP-TMP>
VP|<NP-PP-TMP>

...

VBP
NNS
NNS
NP-SBJ|<CC-NNP>
NP-SBJ|<CC-NNP>
NNP
NNP
VB
VB
VB
VB
NNP
NNP
S
S
S|<,-NP-SBJ-84>
S|<,-NP-SBJ-84>
S|<NP-SBJ-84-VP>
S|<NP-SBJ-84-VP>
NP-SBJ-84
NP-SBJ-84
NNP
NNP
NNP
NNP
VP
VP
VBN
VBN
VP|<NP-NP-ADV>
VP|<NP-NP-ADV>
S
S
NP-SBJ-85
NP-SBJ-85
NP-SBJ-85|<PP-,>
NP-SBJ-85|<PP-,>
NP-SBJ-85|<,-SBAR>
NP-SBJ-85|<,-SBAR>
NP-SBJ-85|<SBAR-,>
NP-SBJ-85|<SBAR-,>
WHNP-1
WHNP-1
WHNP
WHNP
CD
CD
NP|<PP-PP>
NP|<PP-PP>
VBN
VBN
VBG
VBG
NNP
NNP
QP
QP
VB
VB
VBG
VBG
NNS
NNS
VBP
VBP
PP
PP
NP
NP
VB
VB
VP|<NP-TMP-,>
VP|<NP-TMP-,>
VP|<,-CC>
VP|<,-CC>
VP|<CC-NP-1>
VP|<CC-NP-1>
NP
NP
S|<S-TPC-2-,>
S|<S-TPC-2-,>
S-TPC-2
S-TPC-2
S-TPC-2|<,-S>
S-TPC-2|<,-S>
S-TPC-2|<S-,>
S-TPC-2|<S-,>
NN
NN
VP
VP
NP
NP
NP|<''-ADVP>
NP|<''-ADVP>
NNP
NNP
NNP
NNP
NP|<,-NP-LOC>
NP|<,-NP-LOC>
VB
VB
S
S
NNS
NNS
NP
NP
PDT
PDT
VP|<NP-NP-EXT>
VP|<NP-NP-EXT>
VP
VP
VP|<PRT-ADVP-MNR>
VP|<PRT-ADVP-MNR>
RB
RB
VP
VP
S
S
S|<PP-VP>
S|<PP-VP>
VBZ
VBZ
JJ
JJ
SBAR
SBAR
WHPP-3
WHPP-3
NN
NN
VB
VB
IN
IN
NNS
NNS
VP
VP
VBN
VBN
NN
NN
NN
NN
NNS
NNS
JJ
JJ
JJ

CD
NP|<VP-,>
NP|<VP-,>
CD
CD
PP
PP
NP|<,-PP>
NP|<,-PP>
NNP
NNP
NNP
NNP
NP|<CC-NNP>
NP|<CC-NNP>
JJ
JJ
NNS
NNS
NNP
NNP
NP
NP
NP
NP
VB
VB
NP|<NP-TMP-,>
NP|<NP-TMP-,>
NP|<,-PP>
NP|<,-PP>
NP|<PP-S>
NP|<PP-S>
NP|<VBN-CD>
NP|<VBN-CD>
NP|<S-.>
NP|<S-.>
CD
CD
NP-SBJ
NP-SBJ
NP-SBJ|<CD-JJ>
NP-SBJ|<CD-JJ>
NN
NN
VP
VP
VP|<ADJP-PRD-PP-TMP>
VP|<ADJP-PRD-PP-TMP>
VP|<PP-TMP-PP-CLR>
VP|<PP-TMP-PP-CLR>
VP
VP
VP|<S-ADVP-TMP-CLR>
VP|<S-ADVP-TMP-CLR>
ADVP-TMP-CLR
ADVP-TMP-CLR
NNP
NNP
NP
NP
NP
NP
NP|<PP-TMP-,>
NP|<PP-TMP-,>
CD
CD
CD
CD
VP|<,-PP>
VP|<,-PP>
NNP
NNP
NNP
NNP
NP
NP
CD
CD
NNP
NNP
NNP
NNP
NNP
NNP
NN
NN
NP-SBJ-1
NP-SBJ-1
NP-SBJ-1|<NNP-NNPS>
NP-SBJ-1|<NNP-NNPS>
NP-SBJ-1|<NNPS-NN>
NP-SBJ-1|<NNPS-NN>
NNPS
NNPS
NNP
NNP
JJ
JJ
NN
NN
NN
NN
NNP
NNP
FRAG
FRAG
NN
NN
FRAG|<,-SBAR-PRP>
FRAG|<,-SBAR-PRP>
FRAG|<SBAR-PRP-.>
FRAG|<SBAR-PRP-.>
NP-1
NP-1
NP-1|<NNP-NNS>
NP-1|<NNP-NNS>
VB
VB
VP
VP
VBZ
VBZ
NNP
NNP
VB
VB
NP-PRD
NP-PRD
JJ
JJ
NN
NN
NP
NP
NP|<CD-JJS>
NP|<CD-JJS>
VP|<PP-CLR-,>
VP|<PP-CLR-,>
NNS
NNS
VP|<,-VP-

VP-TPC-1|<NP-PP-CLR>
VP-TPC-1|<NP-PP-CLR>
SINV|<VP-NP-SBJ-2>
SINV|<VP-NP-SBJ-2>
SINV|<NP-SBJ-2-.>
SINV|<NP-SBJ-2-.>
NP-SBJ-2
NP-SBJ-2
NNP
NNP
NNP
NNP
NP-SBJ-2|<CC-NP>
NP-SBJ-2|<CC-NP>
NNP
NNP
NNS
NNS
PP-CLR
PP-CLR
JJ
JJ
NN
NN
NNS
NNS
VP
VP
VP|<NP-PP-TMP-CLR>
VP|<NP-PP-TMP-CLR>
NN
NN
VBD
VBD
JJ
JJ
NP-SBJ-1
NP-SBJ-1
NNP
NNP
NNP
NNP
NNP
NNP
NN
NN
JJ
JJ
NP-SBJ-1
NP-SBJ-1
NNP
NNP
NNP
NNP
CD
CD
RB
RB
NP|<VBN-CD>
NP|<VBN-CD>
VBN
VBN
CD
CD
CD
CD
CD
CD
CD
CD
VP
VP
VP|<SBAR-PP-1>
VP|<SBAR-PP-1>
JJ
JJ
VP|<ADJP-PRD-''>
VP|<ADJP-PRD-''>
CD
CD
CD
CD
VB
VB
NP-SBJ
NP-SBJ
JJR
JJR
CD
CD
CD
CD
CD
CD
CD
CD
CD
CD
CD
CD
S-TPC-2
S-TPC-2
VBN
VBN
VBP
VBP
JJ
JJ
NN
NN
NN
NN
RB
RB
S
S
NP-SBJ-2|<NN-NN>
NP-SBJ-2|<NN-NN>
NNS
NNS
VBN
VBN
VP|<PP-TMP-PP>
VP|<PP-TMP-PP>
NNP
NNP
NNP
NNP
VBG
VBG
S-NOM
S-NOM
S-NOM|<CC-S>
S-NOM|<CC-S>
NP-SBJ
NP-SBJ
CD
CD
NNS
NNS
CD
CD
NN
NN
VP
VP
RB
RB
PP-DIR
PP-DIR
CD
CD
PP-DIR|<PP-PP-1>
PP-DIR|<PP-PP-1>
CD
CD
CD
CD
CD
CD
NN
NN
JJ
JJ
NNP
NNP
NNP
NNP
S-TPC-1
S-TPC-1
JJ
JJ
NP|<,-UCP>
NP|<,-U

NN
NN
VP
VP
VP|<PP-,>
VP|<PP-,>
NP
NP
JJ
JJ
NP-SBJ-1
NP-SBJ-1
VP|<S-,>
VP|<S-,>
VB
VB
NNS
NNS
CD
CD
NNS
NNS
NNP
NNP
VP
VP
CD
CD
VBD
VBD
JJ
JJ
JJ
JJ
VP
VP
NNP
NNP
NNS
NNS
NNP
NNP
NNP
NNP
NP-1
NP-1
NN
NN
JJ
JJ
NNP
NNP
NN
NN
NN
NN
VP|<ADJP-PRD-PP-TMP>
VP|<ADJP-PRD-PP-TMP>
JJ
JJ
JJ
JJ
NNS
NNS
VB
VB
NNP
NNP
VP
VP
NP-CLR
NP-CLR
NNP
NNP
NNP
NNP
JJ
JJ
VB
VB
VP
VP
VP|<SBAR-NOM-PP-CLR>
VP|<SBAR-NOM-PP-CLR>
NP-PRD
NP-PRD
JJ
JJ
NNS
NNS
VBD
VBD
CD
CD
NNP
NNP
NNP
NNP
NNP
NNP
NNP
NNP
CD
CD
JJ
JJ
NP|<CD--NONE->
NP|<CD--NONE->
NP|<-NONE--NN>
NP|<-NONE--NN>
NN
NN
NP
NP
NP
NP
NP|<NP-ADV-PP>
NP|<NP-ADV-PP>
NNP
NNP
NNS
NNS
NN
NN
NP-SBJ-1
NP-SBJ-1
NP-SBJ-1|<VBN-NN>
NP-SBJ-1|<VBN-NN>
VB
VB
NP|<VBG-NN>
NP|<VBG-NN>
NNP
NNP
NNP
NNP
JJ
JJ
VBD
VBD
JJ
JJ
NP-SBJ
NP-SBJ
NP-SBJ|<,-UCP>
NP-SBJ|<,-UCP>
NP-SBJ|<UCP-,>
NP-SBJ|<UCP-,>
JJ
JJ
NP|<NNS-NN>
NP|<NNS-NN>
NNS
NNS
UCP|<NN-CC>
UCP|<NN-CC>
JJ
JJ
VBN
VBN
NP-PRD
NP-PRD
NP-PRD|<,-''>
NP-PRD|<,-''>
NP-PRD|<''-CC>
NP-PRD|<''-CC>
VBD
VBD
NN
NN
JJ
JJ
VP|<ADVP-TMP-PP-PRD>
V

VP
ADVP-EXT
ADVP-EXT
VP
VP
SBAR-PRP-PRD
SBAR-PRP-PRD
SBAR-PRP-PRD|<IN-S>
SBAR-PRP-PRD|<IN-S>
VBP
VBP
NN
NN
NNS
NNS
VBN
VBN
NNP
NNP
JJ
JJ
NP-TMP-CLR
NP-TMP-CLR
CD
CD
NP
NP
CD
CD
NP|<,-PRN>
NP|<,-PRN>
CD
CD
CD
CD
CD
CD
CD
CD
CD
CD
CD
CD
JJ
JJ
CD
CD
CD
CD
JJ
JJ
NP
NP
VP
VP
VBD
VBD
VP|<ADVP-DIR-NP>
VP|<ADVP-DIR-NP>
JJ
JJ
NNS
NNS
CD
CD
CD
CD
NN
NN
S
S
ADVP-PRP
ADVP-PRP
RB
RB
VB
VB
CD
CD
VBN
VBN
CD
CD
CD
CD
JJ
JJ
NN
NN
VBZ
VBZ
NNS
NNS
VBD
VBD
VP
VP
VP|<PP-CLR-ADVP-PRP>
VP|<PP-CLR-ADVP-PRP>
ADVP-PRP
ADVP-PRP
JJ
JJ
VP|<ADJP-PRD-ADVP>
VP|<ADJP-PRD-ADVP>
NP-ADV
NP-ADV
NP|<JJS-NN>
NP|<JJS-NN>
NP-ADV|<ADVP-SBAR-1>
NP-ADV|<ADVP-SBAR-1>
VP|<NP-EXT-,>
VP|<NP-EXT-,>
VP|<,-PP-DIR>
VP|<,-PP-DIR>
VP|<PP-DIR-,>
VP|<PP-DIR-,>
NNS
NNS
NP
NP
NNS
NNS
VP|<NP-PP-CLR>
VP|<NP-PP-CLR>
VP|<PP-CLR-PP-1>
VP|<PP-CLR-PP-1>
JJ
JJ
VBZ
VBZ
NN
NN
S
S
S|<ADJP-PRD-SBAR-1>
S|<ADJP-PRD-SBAR-1>
VBG
VBG
VBZ
VBZ
NP-SBJ
NP-SBJ
NN
NN
VB
VB
NP
NP
VP|<PP-DIR-NP-TMP>
VP|<PP-DIR-NP-TMP>
NP
NP
NNS
NNS
CD
CD
CD
CD
CD
CD
NNS
NNS
CD
CD
CD


In [6]:
Unary={}
Binary={}
for i in range(len(grammar.productions())):
    if len(grammar.productions()[i].rhs())==1:
        A=str(grammar.productions()[i]).split(' -> ')[0]
        BC=str(grammar.productions()[i]).split(' -> ')[1].split()[:-1]
        if A in Unary.keys():
            Unary[A].append(BC[0])
        else:
            Unary[A]=[]
            Unary[A].append(BC[0])
    else:
        A=str(grammar.productions()[i]).split(' -> ')[0]
        BC=str(grammar.productions()[i]).split(' -> ')[1].split()[:-1]
        if A in Binary.keys():
            Binary[A].append(BC)
        else:
            Binary[A]=[]
            Binary[A].append(BC)

In [7]:
Binary

{'S': [['NP-SBJ', 'S|<VP-.>'],
  ['NP-SBJ-1', 'S|<VP-.>'],
  ['NP-SBJ', 'NP-PRD'],
  ['S-TPC-1', 'S|<,-NP-SBJ>'],
  ['S-TPC-2', 'S|<,-NP-SBJ>'],
  ['NP-SBJ', 'VP'],
  ['SBAR-ADV', 'S|<,-NP-SBJ>'],
  ['NP-SBJ-2', 'VP'],
  ['NP-SBJ-1', 'VP'],
  ['PP-LOC', 'S|<,-NP-SBJ>'],
  ['NP-SBJ-4', 'VP'],
  ['NP-SBJ-5', 'VP'],
  ['PP-TMP', 'S|<,-NP-SBJ>'],
  ['PP-TMP', 'S|<,-NP-SBJ-6>'],
  ['NP-SBJ-7', 'S|<VP-.>'],
  ['NP-SBJ-8', 'VP'],
  ['``', 'S|<CC-NP-SBJ-1>'],
  ['NP-SBJ-9', 'S|<VP-.>'],
  ['NP-SBJ', 'S|<,-PP>'],
  ['PP', 'S|<,-NP-SBJ-1>'],
  ['ADVP', 'S|<,-NP-SBJ>'],
  ['S', 'S|<:-S>'],
  ['NP-SBJ-10', 'S|<VP-.>'],
  ['NP-SBJ-11', 'S|<VP-.>'],
  ['S', 'S|<,-CC>'],
  ['NP-SBJ-12', 'S|<VP-.>'],
  ['PP-LOC', 'S|<NP-SBJ-VP>'],
  ['ADVP-TMP', 'S|<,-NP-SBJ>'],
  ['SBAR-TMP', 'S|<,-NP-SBJ>'],
  ['CC', 'S|<NP-SBJ-1-VP>'],
  ['PP', 'S|<,-NP-SBJ>'],
  ['NP-TMP', 'S|<,-PP>'],
  ['S', 'S|<,-IN>'],
  ['ADVP-TMP', 'S|<,-PP>'],
  ['S-ADV', 'S|<,-NP-SBJ-1>'],
  ['NP-SBJ', 'S|<ADVP-VP>'],
  ['ADVP-LOC', 'S|<,-

In [8]:
total_dict['VP|<VP-PP>']
total1=list(total)

In [27]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
sent="'I' 'saw' 'John' 'with' 'my' 'eyes'"
sent=sent.split(" ")
print(sent)
nonterms=['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','SYM','TO','UH','VBD','VB','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']
nonterm=len(lhs_1)
score = [ [ [0.0 for y in range(nonterm ) ] for x in range( len(sent) +1 ) ] for z in range(len(sent) +1)]
back = [ [ [None for y in range( nonterm) ] for x in range( len(sent) +1 ) ] for z in range(len(sent) +1)]
a=0
for i in tqdm(range(len(sent))):
    l=sent[i]
    pks=set()
    for A in tqdm(Unary.keys()):
        for pk in prob.keys():    
            if pk.split(' -> ')[1] in sent:
                pks.add(pk.split(' -> ')[0])
        if str(A+" -> "+l) in prob.keys():
            print(A)
            score[i][i+1][lhs_dict[A]] = float(prob[str(A+' -> '+l)])
            print(score[i][i+1][lhs_dict[A]],i)
            a+=1
    print(pks)
    added=1
    print("started")
    while added:
        added=0
        for k in tqdm(Unary.keys()):
            for item in Unary[k]:
                if item in lhs_dict.keys():
                    if score[i][i+1][lhs_dict[item]]>0:
                        p1=prob[str(k)+' -> '+item]*score[i][i+1][lhs_dict[item]]
                        if p1>score[i][i+1][lhs_dict[k]]:
                            score[i][i+1][lhs_dict[k]]=p1
                            back[i][i+1][lhs_dict[k]]=item
                            print(back[i][i+1][lhs_dict[k]])
                            added=1

["'I'", "'saw'", "'John'", "'with'", "'my'", "'eyes'"]


NNP
0.00021254 0
PRP
0.0658508 0
{'NNP', 'PRP$', 'NNS', 'PRP', 'VBD', 'IN'}
started


PRP
PRP
PRP
NNP
PRP
NNP
PRP
NNP
NNP
PRP
NNP
PRP
NNP
PRP
NNP
PRP
NNP
PRP
PRP
PRP
PRP
PRP
PRP
PRP
PRP
NNP
NNP
PRP
NNP
PRP
NNP
NNP
PRP
PRP
PRP
PRP
NNP
PRP
PRP
PRP
PRP
PRP
PRP
NNP
NNP
NNP


VBD
0.00164312 1
{'NNP', 'PRP$', 'NNS', 'PRP', 'VBD', 'IN'}
started


VBD
VBD


...

IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN
IN


PRP$
0.0234987 4
{'NNP', 'PRP$', 'NNS', 'PRP', 'VBD', 'IN'}
started


PRP$
PRP$


NNS
0.000330743 5
{'NNP', 'PRP$', 'NNS', 'PRP', 'VBD', 'IN'}
started


NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS
NNS


In [31]:
for i in range(7):
    print([j for j, e in enumerate(score[i][6]) if e != 0])

[]
[]
[]
[]
[]
[203, 595, 915, 1005, 1300, 1348, 1818, 1918, 2214, 2622, 2762, 2803, 2814, 2828, 3047, 3131, 3186, 3760]
[]


In [28]:
Binary['S']
lhs_dict['S']

3835

In [29]:
print(len(sent))
for span in tqdm(range(2,len(sent)+1)):
    print("span:"+str(span))
    for begin in tqdm(range(len(sent)-span+1)):
        end=begin+span
        print("span: "+str(span)+" begin: "+str(begin)+" end: "+str(end))
        for split in tqdm(range(begin+1,end)):
            print("span: "+str(span)+" begin: "+str(begin)+" end: "+str(end)+" split: "+str(split))
            print("Check in grammar")
            for k in Binary.keys():
                for item in Binary[k]:
                    if begin==0 and end==6:
                        
                        if item[0] in lhs_dict.keys() and item[1] in lhs_dict.keys():
                            
                            p1=score[begin][split][lhs_dict[item[0]]]*score[split][end][lhs_dict[item[1]]]*prob[str(k)+' -> '+item[0]+" "+item[1]]
                            if score[begin][split][lhs_dict[item[0]]]!=0 and score[split][end][lhs_dict[item[1]]]: 
                                print(score[begin][split][lhs_dict[item[0]]])
                                print(score[split][end][lhs_dict[item[1]]])
                            if p1>score[begin][end][lhs_dict[A]]:
                                print(p1)
                                score[begin][end][lhs_dict[A]]=p1
                                back[begin][end][lhs_dict[A]]=(split,item[0],item[1])
                                print(back[begin][end][lhs_dict[A]])
                                print("S: "+k)
                                print(begin,end)
        added=1
        while added:
            added=0
            for k in tqdm(Unary.keys()):
                for item in Unary[k]:
                    if item in lhs_dict.keys():
                        if score[begin][end][lhs_dict[item]]>0:
                            p1=prob[str(k)+' -> '+item]*score[begin][end][lhs_dict[item]]
                            print(p1)
                            if p1>score[begin][end][lhs_dict[k]]:
                                score[begin][end][lhs_dict[k]]=p1
                                back[begin][end][lhs_dict[k]]=item
                                added=1

6


span:2


span: 2 begin: 0 end: 2


span: 2 begin: 0 end: 2 split: 1
Check in grammar


span: 2 begin: 1 end: 3


...

span: 5 begin: 1 end: 6


span: 5 begin: 1 end: 6 split: 2
Check in grammar
span: 5 begin: 1 end: 6 split: 3
Check in grammar
span: 5 begin: 1 end: 6 split: 4
Check in grammar
span: 5 begin: 1 end: 6 split: 5
Check in grammar


span:6


span: 6 begin: 0 end: 6


span: 6 begin: 0 end: 6 split: 1
Check in grammar
span: 6 begin: 0 end: 6 split: 2
Check in grammar
span: 6 begin: 0 end: 6 split: 3
Check in grammar
span: 6 begin: 0 end: 6 split: 4
Check in grammar
span: 6 begin: 0 end: 6 split: 5
Check in grammar


In [13]:
class GrammarTree(object):
    '''
    Tree data structure used to represent the grammar tree output generated by the cky algorithm
    '''
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

    def insertLeft(self, new_node):
            self.left = new_node

    def insertRight(self, new_node):
            self.right = new_node

In [14]:
from collections import deque
def print_level_order(head, queue = deque()):
    if isinstance(head,str):
        print(head)
        return
    print(head.data)
    [queue.append(node) for node in [head.left, head.right] if node]
    if queue:
        print_level_order(queue.popleft(), queue)

In [15]:
def build_tree(start,end,idx,back,non_terms):
    '''
    build_tree() builds tree from the backpointer matrix obtained in the cky() function
    :param start: start index for tree
    :param end: end index for tree
    :param idx: index used to find non_terminal
    :param back: the backpointer matrix
    :param non_terms: a list of non-terminals
    :return:
    '''
    tree = GrammarTree(non_terms[idx])
    node = back[start][end][idx]
    if isinstance(node,tuple):
        split,left_rule,right_rule = node
        tree.insertLeft(build_tree(start,split,left_rule,back,non_terms))
        tree.insertRight(build_tree(split,end,right_rule,back,non_terms))
        return tree
"""
    else:
        if node>0:
            tree.insertLeft(GrammarTree(non_terms[node]))
        return tree
"""

'\n    else:\n        if node>0:\n            tree.insertLeft(GrammarTree(non_terms[node]))\n        return tree\n'

In [16]:
def get_parse_tree(score, back,non_terms):
    '''
    get_parse_tree() calls the build_tree() method
    :param score: score matrix
    :param back: backpointer matrix
    :param non_terms: list of non_terminals
    :return: GrammarTree the final parse tree
    '''
    root_index = score[0][len(score)-1].index(max(score[0][len(score)-1]))
    print(root_index)
    print(score[0][len(score)-1][root_index])
    tree = build_tree(0,len(score)-1,root_index,back,non_terms)
    print(tree)
    return tree

In [30]:
tree=get_parse_tree(score,back,reverse_dict)

0
0.0
None


In [18]:
reverse_dict[0]

'UCP|<CC-NNP>'